# Workshop 1 - Question and Answers
In this workshop, you will learning how to write prompts and feed them into LLMs. You
will also be learning how to use different prompt techniques to improve the response
from the LLM.

## Loading and Explorng the Dataset
The workshop will be using [`facebook/ExploreToM`](https://huggingface.co/datasets/facebook/ExploreToM) dataset from [HuggingFace](https://huggingface.co).

In [1]:
# TODO: Load the following libraries: datasets
from datasets import load_dataset

In [2]:
# Dataset name
dataset_name = "facebook/ExploreToM"

In [3]:
# TODO: load and explore the dataset
dataset = load_dataset(dataset_name)

In [8]:
# TODO: number of rows in the dataset
print(dataset.shape)

# TODO: Keys in the dataset
print(dataset.keys())

# TODO: Feature names
print(dataset['train'].features)

# TODO: Display a single row
print(dataset['train'][100]['story_structure'])
print(dataset['train'][100]['question'])


{'train': (13309, 18)}
dict_keys(['train'])
{'story_structure': Value('string'), 'infilled_story': Value('string'), 'question': Value('string'), 'expected_answer': Value('string'), 'qprop=params': Value('string'), 'qprop=nth_order': Value('int64'), 'qprop=non_unique_mental_state': Value('bool'), 'sprop=is_false_belief_story_1st': Value('bool'), 'sprop=is_false_belief_story_1st_and_2nd': Value('bool'), 'sprop=story_accuracy_1st_raw': Value('float64'), 'sprop=story_accuracy_1st_infilled': Value('float64'), 'sprop=global_idx': Value('int64'), 'param=story_type': Value('string'), 'param=num_stories_total': Value('int64'), 'param=max_sentences': Value('int64'), 'param=num_people': Value('int64'), 'param=num_moves': Value('int64'), 'param=num_rooms': Value('int64')}
Leslie entered the main tent. Leslie left the main tent. Isabella entered the storage trailer. Isabella moved the stuffed rabbit to the wooden chest, which is also located in the storage trailer. Leslie entered the main tent. Isa

In [9]:
# TODO: import pipeline
from transformers import pipeline

## `pipeline`
[`pipeline`](https://huggingface.co/docs/transformers/en/main_classes/pipelines) is an easy to use API to perform inferencing. It provides a wrapper for task-specific pipelines and abstracts most of the complexity by allowing you to focus on the model and the task. 

You can use `pipeline` to perform summarisation, image classification, audio generation, etc. You can find an exhaustive list of `pipeline` task [here](https://huggingface.co/docs/transformers/en/main_classes/pipelines#transformers.pipeline.task).

In [10]:
# TODO: Summarise the text with the pipeline's default model
qna = pipeline('question-answering')


No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


In [15]:
idx = 50
question = dataset['train'][idx]['question']
story = dataset['train'][idx]['story_structure']
story = dataset['train'][idx]['infilled_story']
answer = dataset['train'][idx]['expected_answer']

In [16]:
result = qna(question=question, context=story)
print(f'Context: {story}')
print(f'Question: {question}')
print(f'Generated answer: {result}')
print(f'Acutal answer: {answer}')

Context: The quiet production room was dimly lit by rows of flickering overhead lights, illuminating the space that was cluttered with equipment, scripts, and costume pieces. Toward the back, a large worktable sat against the wall, with shelves lined with plastic storage bins, a leather satchel, and cardboard boxes neatly stacked in the corner. With a quiet entrance, Lachlan joined the scattered equipment and supplies in the production room, the flickering overhead lights serving as a warm welcome. Brooklyn discreetly observed the lone figure reorganizing his supplies from behind a rack of costumes, noticing the script briefly visit a cardboard box before finding a permanent home in Lachlan's leather satchel. As Lachlan's task neared completion, the room welcomed two more figures, Brooklyn and Aubrey slipping in quietly, their eyes adapting to the dim lighting as they took in the organized chaos around them. Secure within a now carefully selected plastic storage bin on the shelves, the

## Manual Inference - Question and Answer
In this section, we will look at what `pipeline` does under the hood to perform its inference. This will give us a better understanding of the major steps involved.

In [17]:
# TODO: load tokenizer
from transformers import AutoTokenizer

## DistilBERT base cased distilled SQuAD
DistilBERT is a small, fast, cheap and light Transformer model trained by distilling BERT base. More details [here](https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).

In [18]:
model_name = "distilbert/distilbert-base-cased-distilled-squad"

In [19]:
# TODO: Create a tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [20]:
# TODO: Encode text
message = "Thou shall not create a machine in the likeness of the human mind."

# encode the message
encoded = tokenizer(message, return_tensors='pt')
print(encoded)


{'input_ids': tensor([[  101,   157, 14640,  4103,  1136,  2561,   170,  3395,  1107,  1103,
          1176,  1757,  1104,  1103,  1769,  1713,   119,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [21]:
enc_text = encoded.input_ids[0]
print(enc_text)

tensor([  101,   157, 14640,  4103,  1136,  2561,   170,  3395,  1107,  1103,
         1176,  1757,  1104,  1103,  1769,  1713,   119,   102])


In [25]:
orig_text = tokenizer.decode(enc_text, skip_special_tokens=True)
print(orig_text)

Thou shall not create a machine in the likeness of the human mind.


In [24]:
for t in enc_text:
   tok = tokenizer.decode(t)
   print(f'{t} = {tok}')

101 = [CLS]
157 = T
14640 = ##hou
4103 = shall
1136 = not
2561 = create
170 = a
3395 = machine
1107 = in
1103 = the
1176 = like
1757 = ##ness
1104 = of
1103 = the
1769 = human
1713 = mind
119 = .
102 = [SEP]


In [26]:
# TODO: Encoding multiple texts
messages = [
   "big black bug bleeds black blood",
   "Thou shall not create a machine in the likeness of the human mind."
]

encoded = tokenizer(messages, return_tensors='pt', padding=True)

In [29]:
# TODO: Decode text
print(encoded)
print('--------------------')
print(encoded.attention_mask)

{'input_ids': tensor([[  101,  1992,  1602, 15430, 24752,  1116,  1602,  1892,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0],
        [  101,   157, 14640,  4103,  1136,  2561,   170,  3395,  1107,  1103,
          1176,  1757,  1104,  1103,  1769,  1713,   119,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
--------------------
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])


In [30]:
print(tokenizer.decode(encoded.input_ids[0]))
print(tokenizer.decode(encoded.input_ids[1]))

[CLS] big black bug bleeds black blood [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
[CLS] Thou shall not create a machine in the likeness of the human mind. [SEP]


## Working with LLMs
Create and instance of the Large Language Model (LLM). We will then create a simple
prompt, tokenize the prompt and feed the tokenized prompt to the LLM. The response
from the LLM will be decoded to human friendly text.

In [32]:
# TODO: Load libraries
from transformers import AutoModelForQuestionAnswering

In [37]:
# load the model
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

In [60]:
# TODO: Load question answer model
idx = 100
question = dataset['train'][idx]['question']
story = dataset['train'][idx]['story_structure']
story = dataset['train'][idx]['infilled_story']
answer = dataset['train'][idx]['expected_answer']
print(answer)



does not know about it


In [49]:
# TODO: Encode context and question
enc_qnc = tokenizer(question, story, return_tensors='pt', padding=True)
print(enc_qnc)


{'input_ids': tensor([[  101,  1327,  1674, 10206,  1341,  1164, 23065,   112,   188,  6369,
          1113,  3782,  6213, 10700,   136,   113,  3520,  1164,  1122,   120,
          1674,  1136,  1221,  1164,  1122,   114,   102,  1109,  3258,  8656,
          1104,  5101,  4204, 15888,  1103, 18652,  5769,  1104,  1103,  3782,
          4745,   117,  9616,   170, 21088,  6814,  1166, 10389,  1104, 12960,
          1116,  1105, 14312,   119,   138,  7859,  5974,  1104,  3618, 13433,
          1105,  6656, 20049, 20619,  1194,  1103,  1586,   117, 11241, 10134,
          1114,  1103,  6531,  3807,  1104,  7053,  1105,  1390,   117,  1112,
          1103,  1480,   112,   188, 24039,  1125,  1198,  4972,  1106,  8362,
         10787,   119,  8521,  5002, 25649,  1154,  1103,  1514,  9459,   117,
          1103,  3807,  1104,  1103,  3782,  1796, 17429, 17032,  1118,  1103,
         11246,  2928,   119,  8521,  4144,  1763,  1103,  5134, 11246, 23841,
          1116,   117, 17862,  1114,  

In [50]:
# TODO: Tokenize the inputs
print(
   tokenizer.decode(enc_qnc.input_ids[0])
)


[CLS] What does Isabella think about Colton ' s belief on festival marketing strategies? ( knows about it / does not know about it ) [SEP] The warm glow of string lights illuminated the vibrant colors of the festival grounds, casting a lively atmosphere over rows of booths and attractions. A faint scent of popcorn and sugar wafted through the air, mingling with the distant sounds of laughter and music, as the night ' s festivities had just begun to unfold. Leslie slipped unnoticed into the main tent, the sounds of the festival outside momentarily muffled by the canvas walls. Leslie slid past the closing canvas flaps, merging with the lively crowd outside, their bright ringmaster attire momentarily camouflaged amidst swirling festivalgoers. The storage trailer stood quietly behind the booths, its door slowly creaking open to admit Isabella. As she entered, the sounds of laughter and music grew duller, replaced by the musty scent of stored supplies and the faint rustle of fabric. As Isab

In [51]:
# query the model 
result = model(enc_qnc.input_ids, enc_qnc.attention_mask)

In [52]:
print(result.keys())
print(result)


odict_keys(['start_logits', 'end_logits'])
QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[  0.2511,  -7.3663,  -8.7323,  -7.2771,  -7.6361,  -8.9226,  -7.7561,
          -9.7753,  -9.9385,  -7.6223,  -8.8285,  -8.2424,  -7.9430,  -8.6275,
          -6.9744,  -4.0213,  -1.6666,  -6.4796,  -5.7669,  -4.8341,  -1.2666,
          -2.7144,  -4.8449,  -7.2371,  -4.7256,  -6.9385,  -8.3332,  -7.8752,
          -8.2298,  -9.6067, -10.9768,  -8.7236, -10.3558, -10.2204,  -9.9687,
          -8.9472,  -9.7640, -10.9021,  -9.9693,  -9.0061, -10.3905, -10.7753,
          -8.3033,  -8.5538,  -7.5416,  -9.2254, -10.2691,  -9.4027, -10.4673,
          -8.8982, -11.5139, -10.7399,  -8.8208, -10.0034,  -8.3106,  -8.8129,
          -9.8705, -11.0240,  -9.4240, -10.8961, -10.7609, -10.3907, -10.5220,
         -11.7206, -11.2209, -11.1130, -11.1538, -11.5679,  -8.7850, -10.5699,
         -10.8330,  -9.8103,  -9.0735, -10.2758, -11.1371,  -8.9919, -10.9623,
          -9.6040, -11.2242,  -9.51

In [53]:
import torch

# find the index of the largest value in start_logit
ans_start = torch.argmax(result.start_logits)
print(ans_start)

tensor(379)


In [54]:
# find the index of the largest value in end_logit
ans_end = torch.argmax(result.end_logits) + 1
print(ans_end)

tensor(383)


In [58]:
enc_ans = enc_qnc.input_ids[0][ans_start: ans_end]
print(enc_ans)
answer = tokenizer.decode(enc_ans)
print(answer)
print('----------------\n')
print(f'Question: {question}')
print(f'Context: {story}')
print(f'Answer: {answer}')

tensor([10206, 22546,   170,  3802])
Isabella mouthed a message
----------------

Question: What does Isabella think about Colton's belief on festival marketing strategies? (knows about it / does not know about it)
Context: The warm glow of string lights illuminated the vibrant colors of the festival grounds, casting a lively atmosphere over rows of booths and attractions. A faint scent of popcorn and sugar wafted through the air, mingling with the distant sounds of laughter and music, as the night's festivities had just begun to unfold. Leslie slipped unnoticed into the main tent, the sounds of the festival outside momentarily muffled by the canvas walls. Leslie slid past the closing canvas flaps, merging with the lively crowd outside, their bright ringmaster attire momentarily camouflaged amidst swirling festivalgoers. The storage trailer stood quietly behind the booths, its door slowly creaking open to admit Isabella. As she entered, the sounds of laughter and music grew duller, rep